# Collaborative Filtering and Comedy! 
------
<img src="images/seinfeld.jpg" width="400" height="400">

## A demo using DataStax Enterprise Analytics, Apache Cassandra, Apache Spark, Python, Jupyter Notebooks, Spark MlLib, and KMeans 

#### Real Dataset: http://eigentaste.berkeley.edu/dataset/ Dataset 2 
#### Rate Jokes: http://eigentaste.berkeley.edu

## Import python packages -- all are required
* Need to tell Jupyter to display with %matplotlib otherwise you will generate the plot but not display it

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.2 pyspark-shell'

In [3]:
import pandas
import cassandra
import pyspark
import re
import os
import matplotlib.pyplot as plt
from IPython.display import IFrame
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#### Helper function to have nicer formatting of Spark DataFrames

In [4]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  10, truncate = False):
    if(truncate):
        pandas.set_option('display.max_colwidth', 100)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

# Apache Cassandra 
<img src="images/cassandralogo.png" width="200" height="200">

## Creating Tables and Loading Tables

## Connect to Apache Cassandra Local Instance

In [5]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.01'])
session = cluster.connect()

### Create Demo Keyspace 

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS jokes 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [7]:
session.set_keyspace('jokes')

### Create table called jokes_table. Our PRIMARY will need to be a unique composite key (userid, jokeid). This will result in an even distribution of the data and allow for each row to be unique. Remember we will have to utilize that PRIMARY KEY in our WHERE clause in any of our CQL queries. 

In [8]:
query = "CREATE TABLE IF NOT EXISTS jokes_table1 \
                                    (userid int, jokeid int, rating float, \
                                     PRIMARY KEY (userid, jokeid))"
session.execute(query)

### Load Joke Dataset
<img src="images/laughing.gif" width="300" height="300">

* Column 1: User id
* Column 2: Joke id
* Column 3: Rating of joke (-10.00 - 10.00) 

### Load Jokes dataset from CSV file (jester_ratings3.csv)
* This is a file I created from the *.dat file and I only have 10,000 rows -- dataset has over 1 million rows

#### Insert all the Joke Rating Data into the Apache Cassandra table `jokes_table`

In [9]:
fileName = 'data/jester_ratings3.csv'
input_file = open(fileName, 'r')

for line in input_file:
    jokeRow = line.split(',')
    query = "INSERT INTO jokes_table1 (userid, jokeid, rating)"
    
    query = query + "VALUES (%s, %s, %s)"
    
    session.execute(query, (int(jokeRow[0]), int(jokeRow[1]) , float(jokeRow[2]) ))

#### Do a select * on joke_table WHERE userid = x to verify that data was loaded into the table

In [10]:
query = 'SELECT * FROM jokes_table1 WHERE userid = 100'
rows = session.execute(query)
for row in rows:
    print (row.userid, row.jokeid, row.rating)

100 5 -0.875
100 7 9.906000137329102
100 8 -0.843999981880188
100 13 8.937999725341797
100 15 -0.968999981880188
100 16 -9.75
100 17 9.593999862670898


## Machine Learning with Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

### Finally time for Apache Spark! 

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [11]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

jokeTable = spark.read.format("org.apache.spark.sql.cassandra").options(table="jokes_table1", keyspace="jokes").load()

print ("Table Row Count: ")
print (jokeTable.count())

Table Row Count: 
10000


#### CFilter with PySpark requires that the ratings not be double/foat but int

In [12]:
showDF(jokeTable)
joke_df = jokeTable.withColumn("rating", jokeTable.rating.cast('int'))
showDF(joke_df)

,userid,jokeid,rating
0,218,5,9.531
1,218,7,-0.594
2,218,8,9.875
3,4,5,-5.812
4,4,7,-4.500
5,4,8,-4.906
6,18,5,-0.438
7,18,7,-7.344
8,18,8,2.375
9,18,13,-2.281


,userid,jokeid,rating
0,218,5,9
1,218,7,0
2,218,8,9
3,4,5,-5
4,4,7,-4
5,4,8,-4
6,18,5,0
7,18,7,-7
8,18,8,2
9,18,13,-2


#### Split dataset into training and testing set 

In [13]:
(training, test) = jokeTable.randomSplit([0.8, 0.2])

training_df = training.withColumn("rating", training.rating.cast('int'))
testing_df = test.withColumn("rating", test.rating.cast('int'))

showDF(training_df)

,userid,jokeid,rating
0,4,7,-4
1,4,8,-4
2,18,5,0
3,18,13,-2
4,18,16,-4
5,18,17,-5
6,18,18,-2
7,18,19,-3
8,18,20,-7
9,18,21,0


### Setup for CFliter with ALS

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [14]:
als = ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="jokeid", ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training_df)

In [15]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(testing_df)

# Generate top 10 joke recommendations for each user
userRecs = model.recommendForAllUsers(10)

showDF(userRecs)

# Generate top 10 user recommendations for each joke
jokeRecs = model.recommendForAllItems(10)

showDF(jokeRecs)


,userid,recommendations
0,148,"[(87, 23.693239212036133), (41, 19.214303970336914), (40, 16.934907913208008), (120, 16.666828155517578), (49, 15.290599822998047), (115, 14.887896537780762), (65, 12.748496055603027), (62, 12.358503341674805), (111, 11.924610137939453), (128, 11.735187530517578)]"
1,243,"[(21, 52.22332000732422), (42, 38.19022750854492), (75, 31.640705108642578), (102, 25.790842056274414), (98, 24.194351196289062), (26, 22.03955841064453), (17, 21.246315002441406), (103, 19.149229049682617), (22, 18.551708221435547), (121, 18.30706024169922)]"
2,251,"[(127, 4.535020351409912), (139, 3.8323304653167725), (56, 1.1971489191055298), (75, 1.0939241647720337), (55, 0.36140280961990356), (146, 0.021901942789554596), (28, -0.0770476907491684), (46, -0.3852476179599762), (54, -0.6195504665374756), (133, -0.6416299343109131)]"
3,85,"[(55, 31.156688690185547), (122, 20.071901321411133), (51, 18.248973846435547), (98, 16.319583892822266), (100, 14.310355186462402), (74, 14.238351821899414), (106, 12.018896102905273), (141, 11.513187408447266), (86, 11.107549667358398), (115, 9.973764419555664)]"
4,137,"[(145, 12.22182846069336), (141, 12.195711135864258), (22, 10.785696029663086), (34, 10.41961669921875), (57, 9.749146461486816), (58, 9.634658813476562), (44, 9.610920906066895), (101, 9.376232147216797), (103, 9.263568878173828), (73, 9.19235897064209)]"
5,65,"[(43, 13.727737426757812), (145, 11.807816505432129), (73, 11.738568305969238), (99, 11.711119651794434), (22, 11.61899185180664), (94, 11.558034896850586), (38, 11.55201530456543), (70, 11.499204635620117), (44, 11.234659194946289), (96, 11.155696868896484)]"
6,53,"[(46, 11.876001358032227), (34, 11.350327491760254), (56, 11.265473365783691), (66, 8.855242729187012), (54, 7.7757887840271), (119, 7.439031600952148), (22, 7.158226490020752), (53, 7.150404453277588), (106, 6.930089473724365), (67, 6.929616928100586)]"
7,133,"[(97, 16.177024841308594), (96, 15.031625747680664), (94, 14.8018217086792), (114, 13.523909568786621), (80, 13.196764945983887), (77, 12.303911209106445), (43, 11.924880027770996), (124, 11.748804092407227), (118, 11.316502571105957), (83, 10.181018829345703)]"
8,155,"[(48, 59.3870849609375), (21, 55.35184097290039), (27, 53.72080612182617), (29, 53.22849655151367), (53, 51.744266510009766), (89, 50.13859176635742), (76, 49.86812973022461), (28, 47.72120666503906), (113, 47.70071029663086), (127, 47.12153244018555)]"
9,108,"[(53, 22.289440155029297), (34, 21.846309661865234), (54, 20.119873046875), (69, 18.41225814819336), (106, 17.773290634155273), (75, 17.03120231628418), (31, 15.104640007019043), (127, 15.039078712463379), (56, 14.251769065856934), (125, 13.990165710449219)]"


,jokeid,recommendations
0,148,"[(84, 29.217105865478516), (26, 21.004850387573242), (81, 20.9891300201416), (27, 19.705514907836914), (136, 17.968904495239258), (253, 14.980398178100586), (77, 13.416237831115723), (241, 11.31454086303711), (218, 11.30172061920166), (240, 10.942242622375488)]"
1,31,"[(187, 43.87614059448242), (175, 32.651702880859375), (185, 31.21963882446289), (205, 26.895610809326172), (239, 22.442604064941406), (215, 21.797924041748047), (216, 20.95880889892578), (184, 20.749011993408203), (12, 20.094078063964844), (197, 19.709922790527344)]"
2,85,"[(27, 41.77655029296875), (241, 26.01925277709961), (195, 21.46881866455078), (218, 19.93073081970215), (15, 18.065715789794922), (26, 18.053173065185547), (81, 16.76680564880371), (58, 16.581417083740234), (67, 15.21080207824707), (183, 13.641006469726562)]"
3,137,"[(27, 28.62961769104004), (84, 18.92493438720703), (183, 17.486547470092773), (175, 17.219995498657227), (178, 15.431453704833984), (81, 15.054351806640625), (195, 13.472312927246094), (205, 13.295065879821777), (184, 11.535163879394531), (185, 11.269432067871094)]"
4,65,"[(175, 50.11051940917969), (185, 39.63606643676758), (187, 32.69991683959961), (204, 30.684829711914062), (84, 28.608972549438477), (205, 27.948909759521484), (28, 27.902297973632812), (215, 23.176734924316406), (81, 22.70570945739746), (183, 19.457942962646484)]"
5,53,"[(155, 51.744266510009766), (171, 40.68080520629883), (204, 33.45423889160156), (185, 33.04222869873047), (187, 30.291749954223633), (175, 29.802959442138672), (84, 27.286861419677734), (12, 27.260211944580078), (205, 23.51573371887207), (108, 22.289440155029297)]"
6,133,"[(178, 22.74848747253418), (155, 21.162763595581055), (187, 19.580524444580078), (123, 13.52707576751709), (253, 12.027945518493652), (183, 11.930315971374512), (216, 11.484613418579102), (75, 11.36410903930664), (129, 10.823883056640625), (166, 10.230624198913574)]"
7,78,"[(178, 26.002111434936523), (155, 19.32936668395996), (84, 17.8879337310791), (256, 17.524728775024414), (195, 13.641348838806152), (253, 12.309162139892578), (136, 12.011627197265625), (2, 11.22333812713623), (61, 11.100119590759277), (187, 10.8145751953125)]"
8,108,"[(84, 53.526065826416016), (187, 36.71051025390625), (205, 30.277080535888672), (185, 26.707712173461914), (26, 24.17721176147461), (253, 23.874042510986328), (136, 23.040666580200195), (165, 22.386451721191406), (38, 18.885421752929688), (240, 17.911317825317383)]"
9,34,"[(183, 33.047325134277344), (26, 27.104074478149414), (187, 24.488361358642578), (175, 23.597742080688477), (108, 21.846309661865234), (241, 19.654104232788086), (215, 18.3605899810791), (185, 18.256343841552734), (55, 17.445220947265625), (12, 16.809118270874023)]"


In [17]:
showDF(userRecs.filter(userRecs.userid == 65))

,userid,recommendations
0,65,"[(43, 13.727737426757812), (145, 11.807816505432129), (73, 11.738568305969238), (99, 11.711119651794434), (22, 11.61899185180664), (94, 11.558034896850586), (38, 11.55201530456543), (70, 11.499204635620117), (44, 11.234659194946289), (96, 11.155696868896484)]"


In [18]:
IFrame(src='images/init94.html', width=700, height=200)

In [19]:
IFrame(src='images/init43.html', width=700, height=200)

In [ ]:
session.execute("""drop table jokes_table1""")